In [84]:
import pandas as pd


In [85]:
import geopandas as gpd

In [86]:
education = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\rough_edu\education.csv")

In [87]:
education.head()

,geo_id,state_name,county,year,adults(18-24)_with_high_school,percent_adults(18-24)_with_high_school,adults(18-24)_with_college_degree,total_median_earnings
0,0500000US01007,Alabama,Bibb,2015,681.0,33.1,25.0,29113
1,0500000US01009,Alabama,Blount,2015,1538.0,33.3,138.0,34549
2,0500000US01015,Alabama,Calhoun,2015,3362.0,28.3,570.0,28449
3,0500000US01017,Alabama,Chambers,2015,1234.0,42.6,114.0,28983
4,0500000US01019,Alabama,Cherokee,2015,739.0,38.3,107.0,30545


In [88]:
# Read the shapefile
gdf = gpd.read_file('C:/Users/athar/OneDrive/Desktop/fulton_ring/cb_2018_us_county_5m/cb_2018_us_county_5m.shp')

In [89]:
print(gdf.head())

  STATEFP COUNTYFP  COUNTYNS        AFFGEOID  GEOID      NAME LSAD  \
0      39      071  01074048  0500000US39071  39071  Highland   06   
1      06      003  01675840  0500000US06003  06003    Alpine   06   
2      12      033  00295737  0500000US12033  12033  Escambia   06   
3      17      101  00424252  0500000US17101  17101  Lawrence   06   
4      28      153  00695797  0500000US28153  28153     Wayne   06   

        ALAND     AWATER                                           geometry  
0  1432479992   12194983  POLYGON ((-83.86976 39.05553, -83.86568 39.247...  
1  1912292630   12557304  POLYGON ((-120.07248 38.50987, -120.07239 38.7...  
2  1701544502  563927612  POLYGON ((-87.62999 30.87766, -87.62946 30.880...  
3   963936864    5077783  POLYGON ((-87.91028 38.57493, -87.90811 38.850...  
4  2099745573    7255476  POLYGON ((-88.94317 31.78421, -88.94336 31.824...  


In [90]:
# keep only the join key + geometry
gdf_slim = gdf[["GEOID", "geometry"]].copy()
gdf_slim["GEOID"] = gdf_slim["GEOID"].astype(str).str.zfill(5)

In [91]:
# create a FIPS code column (last 5 characters of GEO_ID)
education["county_fips"] = education["geo_id"].str[-5:]

In [92]:
# join on FIPS = GEOID
merged = education.merge(
    gdf_slim.rename(columns={"GEOID": "county_fips"}),
    on="county_fips",
    how="left"
)

In [93]:
# Convert merged DataFrame into a GeoDataFrame
gdf = gpd.GeoDataFrame(merged, geometry="geometry", crs="EPSG:4326")

In [94]:
merged.head()
merged.shape

(5544, 10)

In [95]:
merged.to_csv('./final_education.csv', index=False)

In [96]:
merged.head()

,geo_id,state_name,county,year,adults(18-24)_with_high_school,percent_adults(18-24)_with_high_school,adults(18-24)_with_college_degree,total_median_earnings,county_fips,geometry
0,0500000US01007,Alabama,Bibb,2015,681.0,33.1,25.0,29113,01007,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
1,0500000US01009,Alabama,Blount,2015,1538.0,33.3,138.0,34549,01009,"POLYGON ((-86.96336 33.85822, -86.9202 33.8734..."
2,0500000US01015,Alabama,Calhoun,2015,3362.0,28.3,570.0,28449,01015,"POLYGON ((-86.14371 33.70913, -86.12388 33.710..."
3,0500000US01017,Alabama,Chambers,2015,1234.0,42.6,114.0,28983,01017,"POLYGON ((-85.59318 33.10734, -85.23244 33.108..."
4,0500000US01019,Alabama,Cherokee,2015,739.0,38.3,107.0,30545,01019,"POLYGON ((-85.84384 34.24345, -85.82548 34.240..."


In [97]:
# Use a raw string to avoid unicode escape errors in the file path
# Save the GeoDataFrame (not the plain DataFrame) to GeoJSON
gdf.to_file(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\education.geojson', driver='GeoJSON')